In [40]:
# Dependencies 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
from config import weather_api_key
import requests
import time
from scipy.stats import linregress
import gmaps


In [41]:
# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

Beginning Data Retrieval     
-----------------------------


In [42]:
# Create 2000 random latitude and longitude coordinates 
latitudes = np.random.uniform(-90, 90, size = 2000)
longitudes = np.random.uniform(-180, 180, size = 2000)
coordinates = zip(latitudes, longitudes)
coordinates

# Add coordinates to a list 
coordinates_lst = list(coordinates)

In [43]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for coordinate in coordinates_lst:
    city = citipy.nearest_city(coordinate[0],coordinate[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Make an API request for each city 
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_city = city.title()
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']
        # Fill City Data list 
        city_data.append({"City": city_city,
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Description": city_desc,
                         })
        
    #If an error occurs
    except: 
        print("City not found. Skipping")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

hasaki, jp
grand river south east, mu
vysotsk, ru
albany, au
sumter, us
victoria, sc
chuy, uy
satitoa, ws
urumqi, cn
vaini, to
vila nova de milfontes, pt
amderma, ru
albany, au
punta arenas, cl
hilo, us
rikitea, pf
saint-philippe, re
langsa, id
atuona, pf
waipawa, nz
rikitea, pf
bredasdorp, za
barpeta, in
churapcha, ru
buala, sb
ushuaia, ar
kavieng, pg
souillac, mu
punta arenas, cl
arlit, ne
carnarvon, au
maldonado, uy
ushuaia, ar
sao filipe, cv
gardan diwal, af
ilulissat, gl
kathu, za
barrow, us
hermanus, za
busselton, au
moa, cu
punta arenas, cl
ribeira grande, pt
victoria, sc
lorengau, pg
atuona, pf
albany, au
busselton, au
taoudenni, ml
necochea, ar
wahran, dz
vaini, to
salalah, om
barrow, us
palmer, us
palabuhanratu, id
bullhead city, us
east london, za
port alfred, za
yellowknife, ca
rikitea, pf
provideniya, ru
airai, pw
thompson, ca
bluff, nz
atuona, pf
saint-philippe, re
punta arenas, cl
lompoc, us
verkhoyansk, ru
kruisfontein, za
clyde river, ca
karasburg, na
fairbanks, us
tid

In [44]:
# Add the data to a list in a dict then at the DF
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hasaki,JP,2022-08-12 00:04:32,35.7333,140.8333,86.40,76,21,23.55,few clouds
1,Vysotsk,RU,2022-08-12 00:04:33,60.6288,28.5705,62.10,91,97,9.06,overcast clouds
2,Albany,US,2022-08-12 00:04:33,42.6001,-73.9662,77.41,52,60,1.01,broken clouds
3,Sumter,US,2022-08-12 00:00:50,28.7086,-82.0831,82.02,77,100,9.80,overcast clouds
4,Victoria,HK,2022-08-12 00:00:05,22.2855,114.1577,82.38,91,94,8.01,light rain
5,Chuy,UY,2022-08-12 00:04:34,-33.6971,-53.4616,51.33,92,4,6.11,clear sky
6,Urumqi,CN,2022-08-12 00:04:34,43.8010,87.6005,70.75,73,0,4.47,clear sky
7,Vaini,TO,2022-08-12 00:04:34,-21.2000,-175.2000,73.89,62,77,15.52,broken clouds
8,Vila Nova De Milfontes,PT,2022-08-12 00:04:35,37.7238,-8.7828,66.25,92,0,3.56,clear sky
9,Punta Arenas,CL,2022-08-12 00:04:16,-53.1500,-70.9167,33.12,74,80,6.64,broken clouds


In [46]:
# Create CSV file of DF
csv_weather_file = city_data_df.to_csv('weatherpy_database.csv')


In [47]:
# Export CSV file 
city_data_df.to_csv(csv_weather_file, index_label="City_ID")

'City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description\r\n0,Hasaki,JP,2022-08-12 00:04:32,35.7333,140.8333,86.4,76,21,23.55,few clouds\r\n1,Vysotsk,RU,2022-08-12 00:04:33,60.6288,28.5705,62.1,91,97,9.06,overcast clouds\r\n2,Albany,US,2022-08-12 00:04:33,42.6001,-73.9662,77.41,52,60,1.01,broken clouds\r\n3,Sumter,US,2022-08-12 00:00:50,28.7086,-82.0831,82.02,77,100,9.8,overcast clouds\r\n4,Victoria,HK,2022-08-12 00:00:05,22.2855,114.1577,82.38,91,94,8.01,light rain\r\n5,Chuy,UY,2022-08-12 00:04:34,-33.6971,-53.4616,51.33,92,4,6.11,clear sky\r\n6,Urumqi,CN,2022-08-12 00:04:34,43.801,87.6005,70.75,73,0,4.47,clear sky\r\n7,Vaini,TO,2022-08-12 00:04:34,-21.2,-175.2,73.89,62,77,15.52,broken clouds\r\n8,Vila Nova De Milfontes,PT,2022-08-12 00:04:35,37.7238,-8.7828,66.25,92,0,3.56,clear sky\r\n9,Punta Arenas,CL,2022-08-12 00:04:16,-53.15,-70.9167,33.12,74,80,6.64,broken clouds\r\n10,Hilo,US,2022-08-12 00:03:02,19.7297,-155.09,90.28,80,89,4.0,overcast clo